In [ ]:
from P2 import *
import numpy as np
import os, math, time, argparse
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [ ]:
data, labels, sub_info, form_names, feat_names  = load_dataset(data_path = "P2/data")
num_subs = len(np.unique(sub_info[:, 0])) + 1
num_feats = data.shape[1]
num_forms = len(form_names) + 1
filter_feat_count = 100

In [ ]:
sub_train_acc = np.zeros(num_subs)
sub_class_train = np.zeros((num_subs, num_forms))
sub_test_acc = np.zeros(num_subs)
sub_class_test = np.zeros((num_subs, num_forms))
overall_train_mat = np.zeros((num_forms, 1))
overall_test_mat = np.zeros((num_forms, 1))

In [ ]:
train_data, train_label, test_data, test_label = split_data(data, labels, sub_info, 10)
train_data, train_min, train_max = normalize_data(train_data)
test_data, test_min, test_max = normalize_data(test_data)